# AI Platform - External Model Serving

This notebook uses this AI Platform Notebook to train a TensorFlow model (locally) with the data in BigQuery table `<PROJECT_ID>.digits.digits_prepped`.  This model is then saved and AI Platform clients are used to upload the model and deploy it to an endpoint for online predictions.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`

**Overview**

<img src="architectures/statmike-mlops-04.png">

---
## Setup

Prepare TensorFlow:

In [51]:
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf

Setup Parameters

In [66]:
PROJECT_ID='statmike-mlops'
REGION='us-central1'

BQDATASET_ID='digits'
BQTABLE_ID='digits_prepped'

MODEL_DIR='gs://{}/digits/keras'.format(PROJECT_ID)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

BATCH_SIZE = 30

MODEL_NAME='MODEL_KERAS-DIGITS'
ENDPOINT_NAME='ENDPOINT_KERAS-DIGITS'
params = {"MODEL_DIR":MODEL_DIR}
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'
DEPLOY_COMPUTE='n1-standard-4'

Setup AI Platform Python Clients
- https://googleapis.dev/python/aiplatform/latest/index.html

In [53]:
from google.cloud import aiplatform

API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

---
## Prepare Data Connection

Retrieve the Schema info from BigQuery Information Schema via the Storage API:
- https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas

In [54]:
from google.cloud import bigquery
bqclient = bigquery.Client()
bqjob = bqclient.query(
"""
SELECT * FROM `"""+BQDATASET_ID+""".INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
WHERE TABLE_NAME = '"""+BQTABLE_ID+"""' """
)
schema = bqjob.result().to_dataframe()
schema

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,statmike-mlops,digits,digits_prepped,p0,p0,FLOAT64,None
1,statmike-mlops,digits,digits_prepped,p1,p1,FLOAT64,None
2,statmike-mlops,digits,digits_prepped,p2,p2,FLOAT64,None
3,statmike-mlops,digits,digits_prepped,p3,p3,FLOAT64,None
4,statmike-mlops,digits,digits_prepped,p4,p4,FLOAT64,None
...,...,...,...,...,...,...,...
62,statmike-mlops,digits,digits_prepped,p62,p62,FLOAT64,None
63,statmike-mlops,digits,digits_prepped,p63,p63,FLOAT64,None
64,statmike-mlops,digits,digits_prepped,target,target,INT64,None
65,statmike-mlops,digits,digits_prepped,target_OE,target_OE,STRING,None


Use the the table schema to prepare the TensorFlow Model:
- Omit unused columns
- Create `feature_columns` for the model
- Define the `dtypes` for TensorFlow

In [55]:
OMIT = ['target_OE','SPLITS']

selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != 'target':
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

from tensorflow.python.framework import dtypes
output_types = schema[~schema.column_name.isin(OMIT)].data_type.tolist()
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

Define a function that remaps the input data for TensorFlow into features, target and one_hot encodes the `target`:

In [56]:
def transTable(row_dict):
    target=row_dict.pop('target')
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

Setup TensorFlow_IO client > session > table + table.map
- https://www.tensorflow.org/io/api_docs/python/tfio/bigquery/BigQueryClient

In [57]:
client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TRAIN'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
train = table.shuffle(100000).batch(BATCH_SIZE)

In [58]:
client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TEST'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
test = table.batch(BATCH_SIZE)

Review a single batch of the train data:

In [59]:
for a, b in train.take(1):
    columns=list(a.keys())
    print('columns: ',columns)
    print('target: ',b)

columns:  ['p0', 'p1', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p2', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p3', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p4', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p5', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p6', 'p60', 'p61', 'p62', 'p63', 'p7', 'p8', 'p9']
target:  tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0

---
## Train the Model

Define the Model:

In [60]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()],outputs=tf.keras.layers.Dense(10,activation=tf.nn.softmax)(feature_layer_outputs))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
#tf.keras.utils.plot_model(model,show_shapes=True, show_dtype=True)

In [61]:
#model.summary()

Fit the Model:

In [62]:
history = model.fit(train,epochs=25)

Epoch 1/25
49/49 [==============================] - 3s 8ms/step - loss: 5.4327 - accuracy: 0.3615
Epoch 2/25
49/49 [==============================] - 1s 8ms/step - loss: 0.6904 - accuracy: 0.8120
Epoch 3/25
49/49 [==============================] - 1s 7ms/step - loss: 0.4632 - accuracy: 0.8692
Epoch 4/25
49/49 [==============================] - 1s 9ms/step - loss: 0.3334 - accuracy: 0.8935
Epoch 5/25
49/49 [==============================] - 1s 10ms/step - loss: 0.2654 - accuracy: 0.9265
Epoch 6/25
49/49 [==============================] - 1s 5ms/step - loss: 0.2185 - accuracy: 0.9378
Epoch 7/25
49/49 [==============================] - 1s 9ms/step - loss: 0.2198 - accuracy: 0.9306
Epoch 8/25
49/49 [==============================] - 1s 10ms/step - loss: 0.1562 - accuracy: 0.9490
Epoch 9/25
49/49 [==============================] - 1s 9ms/step - loss: 0.1232 - accuracy: 0.9576
Epoch 10/25
49/49 [==============================] - 1s 10ms/step - loss: 0.1234 - accuracy: 0.9741
Epoch 11/25
49/4

Evaluate the model with the test data:

In [63]:
loss, accuracy = model.evaluate(test)

12/12 [==============================] - 2s 11ms/step - loss: 0.1230 - accuracy: 0.9491


Create Prediction from a batch of the test data:

In [64]:
model.predict(test.take(1))

array([[5.02995135e-07, 9.73487258e-01, 1.84339240e-06, 6.80787722e-03,
        2.17787274e-05, 1.29038608e-05, 3.81712191e-08, 1.42034696e-05,
        2.89012387e-04, 1.93645731e-02],
       [1.04811237e-09, 4.58381295e-01, 5.80528103e-06, 1.38176896e-03,
        7.20336247e-05, 7.80577200e-11, 5.48565549e-05, 1.55116959e-05,
        5.40088713e-01, 5.36114797e-08],
       [4.23839097e-09, 9.77244258e-01, 7.73140769e-08, 7.55626525e-06,
        2.24697925e-02, 1.06107381e-08, 1.76585431e-07, 2.71904423e-06,
        2.69031239e-04, 6.41386032e-06],
       [5.41244723e-11, 9.99505639e-01, 3.32350640e-12, 1.76868454e-07,
        4.40015050e-04, 2.05316292e-10, 7.37809103e-09, 1.90663388e-06,
        4.72779102e-05, 4.97291967e-06],
       [4.18799440e-11, 9.99966979e-01, 7.42060857e-10, 8.54187931e-07,
        1.00361985e-05, 9.39127531e-09, 1.03046371e-09, 2.45011389e-08,
        2.80012841e-06, 1.92962107e-05],
       [1.95367070e-10, 9.94653940e-01, 7.72150486e-08, 6.16667285e-06,
   

---
## Save the model:

In [65]:
model.save(MODEL_DIR)

INFO:tensorflow:Assets written to: gs://statmike-mlops/digits/keras/assets


---
## Upload the Model to AI Platform

Create a client to the Model Service, define the Model, and upload the model:

In [67]:
clients['model'] = aiplatform.gapic.ModelServiceClient(client_options=client_options)

MODEL = {
    "display_name": MODEL_NAME,
    "metadata_schema_uri": "",
    "artifact_uri": MODEL_DIR,
    "container_spec": {
        "image_uri": DEPLOY_IMAGE,
        "command": [],
        "args": [],
        "env": [],
        "ports": [{"container_port": 8080}],
        "predict_route": "",
        "health_route": ""
    }
}

uploaded_model = clients['model'].upload_model(parent=PARENT, model=MODEL)

Retrieve the model information and view the name and display name:

In [68]:
model_info = clients['model'].get_model(name=uploaded_model.result(timeout=180).model)
model_info.display_name, model_info.name

('MODEL_KERAS-DIGITS',
 'projects/691911073727/locations/us-central1/models/945703145189670912')

---
## Create the AI Platform Endpoint

Create a client to the Endpoint Service and use it to create the endpoint:

In [69]:
clients['endpoint'] = aiplatform.gapic.EndpointServiceClient(client_options=client_options)

endpoint = clients['endpoint'].create_endpoint(parent=PARENT, endpoint={"display_name": ENDPOINT_NAME})

Retrieve the endpoint information and view the name and display name:

In [70]:
endpoint_info = clients['endpoint'].get_endpoint(name=endpoint.result(timeout=180).name)
endpoint_info.display_name, endpoint_info.name

('ENDPOINT_KERAS-DIGITS',
 'projects/691911073727/locations/us-central1/endpoints/6521933540060299264')

---
## Deploy the Model to the AI Platform Endpoint

In [71]:
DMODEL = {
        "model": model_info.name,
        "display_name": 'DEPLOYED_'+MODEL_NAME,
        "dedicated_resources": {
            "min_replica_count": 1,
            "max_replica_count": 1,
            "machine_spec": {
                    "machine_type": DEPLOY_COMPUTE,
                    "accelerator_count": 0,
                }
        }   
}

TRAFFIC = {
    '0' : 100
}

dmodel = clients['endpoint'].deploy_model(endpoint=endpoint_info.name, deployed_model=DMODEL, traffic_split=TRAFFIC)

Retrieve the deployed model information from the endpoint:

In [77]:
clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models

[id: "7863935860272529408"
model: "projects/691911073727/locations/us-central1/models/945703145189670912"
display_name: "DEPLOYED_MODEL_KERAS-DIGITS"
create_time {
  seconds: 1618274179
  nanos: 458109000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
]

---
## Predictions

Create a client to the prediction service:

In [78]:
clients['prediction'] = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

Setup an observation for prediction:

In [80]:
%%bigquery pred
SELECT *
FROM `statmike-mlops.digits.digits_prepped`
WHERE splits='TEST'

In [81]:
pred.head(1)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p57,p58,p59,p60,p61,p62,p63,target,target_OE,SPLITS
0,0.0,0.0,0.0,0.0,13.0,16.0,6.0,0.0,0.0,0.0,...,0.0,0.0,1.0,12.0,14.0,5.0,0.0,1,Odd,TEST


In [82]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

Request preciction from the prediction service:

In [83]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

response = clients['prediction'].predict(endpoint=endpoint_info.name, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

In [85]:
response.predictions

[[5.02994226e-07, 0.973487377, 1.8433891e-06, 0.00680787489, 2.17787274e-05, 1.29038508e-05, 3.81712155e-08, 1.42034578e-05, 0.000289012154, 0.0193645582]]

In [86]:
import numpy as np
np.argmax(response.predictions[0])

1

# Remove Resources
- undeploy-model
- remove endpoint
- remove model
- delete model files

Undeploy Model:

In [87]:
dmodel = clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models[0].id
clients['endpoint'].undeploy_model(endpoint=endpoint_info.name, deployed_model_id=dmodel)

Delete Endpoint:

In [88]:
clients['endpoint'].delete_endpoint(name=endpoint_info.name)

Remove Model:

In [89]:
clients['model'].delete_model(name=model_info.name)

Delete Model Files:

In [92]:
from google.cloud import storage
gcs = storage.Client()

path = gcs.bucket(PROJECT_ID)
blobs = path.list_blobs(prefix='digits/keras')
for blob in blobs:
    blob.delete()